In [2]:
############################################################
# Kim, Myeongkyun, GIST, BioRobotics
# maengkyun@gm.gist.ac.kr
############################################################ 
# Quadrotor Simplified Dynamics
############################################################
import sympy as sp
import numpy as np

In [182]:
#Force & Torque Relation between Quadrotor and each rotor
Ft=sp.symbols('F_T') 
tau_phi=sp.symbols('tau_phi') 
tau_theta=sp.symbols('tau_theta') 
tau_psi=sp.symbols('tau_psi') 
L=sp.symbols('L')
A=sp.symbols('A')
m=sp.symbols('m')

T = sp.Matrix([
    [1, 1, 1, 1], 
    [L/np.sqrt(2), -L/np.sqrt(2), -L/np.sqrt(2), L/np.sqrt(2)],
    [-L/np.sqrt(2), -L/np.sqrt(2), L/np.sqrt(2), L/np.sqrt(2)],
    [A, -A, +A, -A]
])
#Rotor variable
F1=sp.symbols('F_1')
F2=sp.symbols('F_2')
F3=sp.symbols('F_3')
F4=sp.symbols('F_4')

#Command Force and Torque of Quadrotorin Body coordinates
M_c=sp.Matrix([ [Ft],
                [tau_phi],
                [tau_theta],
                [tau_psi]])
#Force of each rotor in Body coordinates
M=sp.Matrix([   [F1],
                [F2],
                [F3],
                [F4]])
        
#Force of each rotor in Body coordinates
f = sp.Inverse(T)@M_c
#Force and Torque of Quadrotor in Body coordinates
F=T@M




In [180]:
#Quadrotor Simplified Dynamics
# w : angular velocity in the body frame
# I : inertia
# C: Relation of angular velocity from Navigation Frame  to Body Frame 
# eta : euler angle of quadrotor in the navigation frame

#Euler - Body Relation
# v : linear velocity in the body frame
# phi,theta, psi : euler angle in the navigation frame
# R: Rotation Matrix X-Y-Z order
# p_pp: acceleration of quadrotor in the navigation frame
phi=sp.symbols('phi')
theta=sp.symbols('theta')
psi=sp.symbols('psi')
vx_p=sp.symbols('vx_p')
vy_p=sp.symbols('vy_p')
vz_p=sp.symbols('vz_p')
m=sp.symbols('m')
g=sp.symbols('g')

v_p = sp.Matrix([ [0],
                [0],
                [vz_p]])
g=sp.Matrix([
        [0],
        [0],
        [9.81]
        ])
#x_pp = -1/m*(sp.cos(phi)*sp.sin(theta)*sp.cos(psi)+sp.sin(phi)*sp.sin(psi))*vx_p
#y_pp = -1/m*(sp.cos(phi)*sp.sin(theta)*sp.sin(psi)-sp.sin(phi)*sp.cos(psi))*vy_p
#z_pp = g-1/m*(sp.cos(phi)*sp.cos(theta))*vz_p

R_x = sp.Matrix([   [1,           0,            0],
                    [0, sp.cos(phi), -sp.sin(phi)],
                    [0, sp.sin(phi),  sp.cos(phi)]])

R_y = sp.Matrix([   [sp.cos(theta), 0, sp.sin(theta)],
                    [0,             1,            0],
                    [-sp.sin(theta), 0,  sp.cos(theta)]])

R_z = sp.Matrix([   [sp.cos(psi), -sp.sin(psi),       0],
                    [sp.sin(psi),  sp.cos(psi),       0],
                    [0,                      0,       1]])
R=R_z@R_y@R_x
#Force in the Navigation frame
p_pp = R@v_p-g
p_pp 


phi_p=sp.symbols('phi_p')
theta_p=sp.symbols('theta_p')
psi_p=sp.symbols('psi_p')

I_xx=sp.symbols('I_xx')
I_yy=sp.symbols('I_yy')
I_zz=sp.symbols('I_zz')
T_body=sp.Matrix([
                [tau_phi],
                [tau_theta],
                [tau_psi]
                ])
w_p=R_x.T@R_y.T@sp.Matrix([[0],[0],[psi_p]])+R_x.T@sp.Matrix([[0],[theta_p],[0]])+sp.Matrix([[phi_p],[0],[0]])
I=sp.Matrix([
            [I_xx,  0,     0],
            [0,  I_yy,     0],
            [0,     0,  I_zz]
            ])
C=sp.Matrix([
            [1,    0,  -sp.sin(theta)],
            [0, sp.cos(phi),    sp.sin(phi)*sp.cos(theta)],
            [0, -sp.sin(phi),   sp.cos(phi)*sp.cos(theta)]
            ])
eta_p = sp.Matrix([[phi_p],[theta_p],[psi_p]])
#Torque in the Navigation frame
eta_pp=sp.Inverse(I)@sp.Inverse(C)@T_body
eta_pp




Matrix([
[tau_phi/I_xx + tau_psi*sin(theta)*cos(phi)/(I_xx*(sin(phi)**2*cos(theta) + cos(phi)**2*cos(theta))) + tau_theta*sin(phi)*sin(theta)/(I_xx*(sin(phi)**2*cos(theta) + cos(phi)**2*cos(theta)))],
[                                                                                -tau_psi*sin(phi)/(I_yy*(sin(phi)**2 + cos(phi)**2)) + tau_theta*cos(phi)/(I_yy*(sin(phi)**2 + cos(phi)**2))],
[                                     tau_psi*cos(phi)/(I_zz*(sin(phi)**2*cos(theta) + cos(phi)**2*cos(theta))) + tau_theta*sin(phi)/(I_zz*(sin(phi)**2*cos(theta) + cos(phi)**2*cos(theta)))]])

In [186]:
# Throttle Relation from Body Frame to Navigation Frame 
x_pp = sp.symbols('x_pp')
y_pp = sp.symbols('y_pp')
z_pp = sp.symbols('z_pp')
phi_pp = sp.symbols('phi_pp')
theta_pp = sp.symbols('theta_pp')
psi_pp = sp.symbols('psi_pp')
eta_pp = sp.Matrix([[phi_pp],[theta_pp],[psi_pp]])

#Z-axis Force in the Navigation Frame 
F = sp.Matrix([ [0],
                [0],
                [z_pp]])
#Body Frame
p_body_pp = R.T@F
p_body_pp

torque = C@eta_pp
torque




Matrix([
[                    phi_pp - psi_pp*sin(theta)],
[psi_pp*sin(phi)*cos(theta) + theta_pp*cos(phi)],
[psi_pp*cos(phi)*cos(theta) - theta_pp*sin(phi)]])

In [ ]:
# Omega1 = sp.symbols('Omega_1')
# Omega2 = sp.symbols('Omega_2')
# Omega3 = sp.symbols('Omega_3')
# Omega4 = sp.symbols('Omega_4')
# dp=sp.symbols('d_p')
# bp=sp.symbols('b_p')
# F1 = dp*Omega1*Omega1
# F2 = dp*Omega2*Omega2
# F3 = dp*Omega3*Omega3
# F4 = dp*Omega4*Omega4
# F1=sp.symbols('F_1')
# F2=sp.symbols('F_2')
# F3=sp.symbols('F_3')
# F4=sp.symbols('F_4')
#A  = bp*Omega4*Omega4/F4